### Let's develop a chatbot based on groq llms using langchain and deploy it on haggingface 

In [13]:
import os 
import warnings
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceEndpoint

warnings.filterwarnings("ignore")

In [ ]:
#Test wether api key is there or not
os.environ["HUGGINGFACE_API_TOKEN"]

In [5]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant", #Check groq website for other llm ids
    temperature = 0.6 
)

In [7]:
#test
test_prompt = "Hello, do you hear me ?"
response = groq_llm.invoke(test_prompt)
print (response.content)

Hello. I can see that you've initiated a conversation. I'm here to assist you with any questions or topics you'd like to discuss. How can I help you today?


In [10]:
#test 2
groq_llm.predict(test_prompt)

'Hello, yes I hear you. Is there something I can help you with or would you like to chat?'

In [37]:
flan_t5_llm = HuggingFaceEndpoint(huggingfacehub_api_token = os.environ["HUGGINGFACE_API_TOKEN"],
                                repo_id="google/flan-t5-large", 
                                task = 'text-generation',
                                temperature = 0)

In [ ]:
flan_t5_llm.invoke("hello")

### Prompt Templates nd langchain chains

In [45]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [43]:
prompt_template = PromptTemplate(input_variables = ["word"],
                                 template = "What are 5 synonyms of word {word}")
prompt_template.format(word = "impossible")

'What are 5 synonyms of word impossible'

In [48]:
chain = LLMChain(llm = groq_llm, prompt = prompt_template)
synonyme_response = chain.run("impossible")
print (synonyme_response)

Here are 5 synonyms for the word "impossible":

1. Unfeasible
2. Inconceivable
3. Unthinkable
4. Unachievable
5. Unattainable


In [49]:
print (chain.run("possible"))

Here are 5 synonyms of the word "possible":

1. Feasible
2. Probable
3. Viable
4. Plausible
5. Likely


### Combining multiple chains using simple sequential chain 

In [18]:
import os 
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import SimpleSequentialChain

In [6]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant",
    temperature = 0.6,
)

In [10]:
first_prompt = PromptTemplate(input_variables =["word"], 
                             template = "What are one synonym of word {word}, return the synonyme and nothing else" )
first_chain = LLMChain(llm = groq_llm, prompt = first_prompt)

In [14]:
first_chain.run(word = "impossible")

'Infeasible'

In [20]:
second_prompt = PromptTemplate(input_variables = ["synonyme"],
                                template = "What is the translation of this word {synonyme} in french")
second_chain = LLMChain(llm = groq_llm, prompt = second_prompt)

In [17]:
print(second_chain.run(synonyme = "Infeasible"))

The translation of the word "Infeasible" in French is "Impossible à réaliser" or "Inexécutable". However, a more common and idiomatic translation would be "Inabordable" or "Non réalisable".

Alternatively, you can also use the word "Iréalisable" which is more literal and direct translation of the word "Infeasible".

It's worth noting that the context in which the word is used can affect the choice of translation.


In [22]:
#Sequantial chain 
simple_sequence_chain = SimpleSequentialChain(chains = [first_chain, second_chain])

In [24]:
output = sequence_chain.run("nice")
print (output)

The translation of the word "Friendly" in French is:

- Amical (masculine)
- Amicale (feminine)

However, if you're referring to a friendly atmosphere or a friendly place, you can also use:

- Chaleureux (warm, welcoming)
- Accueillant (welcoming, hospitable)
- Amical (friendly, in a general sense)

If you're talking about a friendly game or competition, you can use:

- Amical (friendly, in a sporting context)

Keep in mind that the context and the situation will influence the correct translation and usage of the word.


### Sequential chains 

In [36]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

In [48]:
first_template = PromptTemplate(input_variables = ["proverb"], 
                                template = "Complete this proverb {proverb}, return the complete proverb only nothin more.")
first_chain = LLMChain(llm = groq_llm, prompt = first_template, output_key ="proverb_completed")

In [27]:
print (first_chain.run("He who plays with the sword"))

He who plays with the sword, must be prepared to be wounded.


In [49]:
second_template = PromptTemplate (input_variables = ["proverb_completed"], 
                                  template = """
                                                Translate this proverbe <{proverb_completed}> to french, 
                                                return only one translation and nothing more.
                                             """)
second_chain = LLMChain(llm = groq_llm, prompt = second_template, output_key = "translation")

In [35]:
print (second_chain.run("He who plays with the sword, must be prepared to be wounded."))

The translation of "He who plays with the sword, must be prepared to be wounded" in French is:

"Qui joue avec l'épée, doit être prêt à se faire blesser."

However, a more idiomatic translation would be:

"Qui se fait l'amitié de l'épée, doit se faire la guerre."

This translation uses a more poetic and idiomatic expression, which conveys the same idea as the original phrase.


In [50]:
sequential_chain = SequentialChain(chains = [first_chain, second_chain],
                input_variables = ["proverb"],
                output_variables = ["proverb_completed", "translation"])

In [51]:
sequential_chain({"proverb": "He who plays with the sowrd"})

{'proverb': 'He who plays with the sowrd',
 'proverb_completed': 'He who plays with the sword, gets cut.',
 'translation': '"Qui se livre à l\'épée, se fait couper."'}